In [2]:
import paho.mqtt.client as mqtt
import time
import threading

# MQTT settings
BROKER = "158.39.162.129"
PORT = 1883

ROLE = "leader"  # Define this JetBot as the leader
LEADER_IP = "192.168.1.12"  # Default IP for leader

if ROLE == "leader":
    print(f"This JetBot is the Leader with IP: {LEADER_IP}")
else:
    print("Error: Role not defined properly")
    exit()

TOPIC_LEADER = "platoon/leader"
TOPIC_FOLLOWERS = "platoon/followers"
TOPIC_STATUS = "platoon/status"
TOPIC_SYNC = "platoon/sync"
TOPIC_HEARTBEAT = "platoon/heartbeat"

# Initialize the MQTT client
client = mqtt.Client()

# Callback when connected to the broker
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker successfully.")
        client.subscribe([(TOPIC_FOLLOWERS, 0), (TOPIC_STATUS, 0)])
        client.publish(TOPIC_LEADER, "move", retain=True)
    else:
        print(f"Connection failed with code {rc}")

# Callback when disconnected from the broker
def on_disconnect(client, userdata, rc):
    if rc != 0:
        print("Unexpected disconnection. Reconnecting...")
    else:
        print("Disconnected cleanly.")

# Callback for received messages
def on_message(client, userdata, msg):
    print(f"Received on {msg.topic}: {msg.payload.decode()}")

# Heartbeat thread to notify followers the leader is online
def send_heartbeat():
    try:
        while True:
            client.publish(TOPIC_HEARTBEAT, "leader_online")
            time.sleep(10)  # Adjust heartbeat interval as needed
    except Exception as e:
        print(f"Heartbeat error: {e}")

# Command and synchronization loop
def command_loop():
    try:
        while True:
            # Send commands to followers
            command = input("Enter command for followers (e.g., 'move', 'stop', 'move right', 'move left','speed up','slow down'): ")
            client.publish(TOPIC_LEADER, command)
            print(f"Command '{command}' sent to followers.")

            # Send sync signals to followers
            sync_signal = input("Send sync signal (e.g., 'sync', 'align') or leave blank: ")
            if sync_signal:
                client.publish(TOPIC_SYNC, sync_signal)
                print(f"Sync signal '{sync_signal}' sent to followers.")

            time.sleep(1)  # Add a short delay between iterations
    except KeyboardInterrupt:
        print("Exiting command loop...")
    finally:
        client.loop_stop()
        client.disconnect()

# Setup MQTT client callbacks
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message

# Connect to the broker
try:
    client.connect(BROKER, PORT, keepalive=60)
except Exception as e:
    print(f"Error connecting to broker: {e}")
    exit()

# Start MQTT loop and heartbeat thread
client.loop_start()
heartbeat_thread = threading.Thread(target=send_heartbeat, daemon=True)
heartbeat_thread.start()

# Start the command loop
command_loop()


This JetBot is the Leader with IP: 192.168.1.12
Connected to broker successfully.
Received on platoon/status: {'position': 9.572388322901341}
Received on platoon/followers: Acknowledged: move
Received on platoon/status: {'position': 8.706315696763774}
Received on platoon/status: {'position': 6.553450327885012}


Enter command for followers (e.g., 'move', 'stop', 'move right', 'move left','speed up','slow down'):  move


Command 'move' sent to followers.
Received on platoon/followers: Acknowledged: move


Send sync signal (e.g., 'sync', 'align') or leave blank:  


Received on platoon/status: {'position': 8.38625180356296}
Received on platoon/status: {'position': 7.7593347798877375}


Enter command for followers (e.g., 'move', 'stop', 'move right', 'move left','speed up','slow down'):  move left


Command 'move left' sent to followers.
Received on platoon/followers: Acknowledged: move left
Received on platoon/status: {'position': 0.7014014912589983}
Received on platoon/status: {'position': 6.105077083470607}
Received on platoon/status: {'position': 2.272129925449061}
Received on platoon/status: {'position': 5.450095496320242}
Received on platoon/status: {'position': 7.523456544586034}
Exiting command loop...
Disconnected cleanly.
